In [ ]:
import pandas as pd
import numpy as np
import os
import sys
import time
import shutil
import random
from glob import glob
import gc
import cv2
import matplotlib.pyplot as plt
from tqdm import tqdm
import warnings
warnings.simplefilter('ignore')

import albumentations as A
from albumentations.pytorch import ToTensorV2

from scipy.special import softmax

import torch
import torchvision.utils as vutils
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [ ]:
import sys
sys.path.append('../input/timm-pytorch-image-models/pytorch-image-models-master')
import timm

In [ ]:
SEED=777
random.seed(SEED)
os.environ['PYTHONHASHSEED'] = str(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
torch.cuda.manual_seed(SEED)

In [ ]:
DEBUG = False#False
N_BINS = 10
N_TTA_384 = 4
N_TTA_224 = 6
BATCH_SIZE = 32

In [ ]:
# 10 bins
BIN2PAW_LIST_pawpularity = np.array([5.15555556,
                                     16.57213439,
                                     25.87837352,
                                     35.01965812,
                                     44.98826119,
                                     55.20275862,
                                     64.98190045,
                                     74.85964912,
                                     84.91907514,
                                     98.53865979])

In [ ]:
MODEL_PATHS = glob('../input/swinl224384jointregbin/*/*.pth')
#MODEL_PATHS = np.array(MODEL_PATHS)[np.array(['Seed777' in _ for _ in MODEL_PATHS])].tolist()

ALPHAS = [float(_.split('-')[-1][0:-4]) for _ in MODEL_PATHS]
INPUT_SIZES = [int(m_path.split('/')[-2].split('SwinL')[1].split('-')[0]) for m_path in MODEL_PATHS]

bin2paw_lists = [BIN2PAW_LIST_pawpularity]*len(MODEL_PATHS)

In [ ]:
print(ALPHAS)
print(MODEL_PATHS)
print(INPUT_SIZES)
print(len(ALPHAS), len(MODEL_PATHS))

In [ ]:
if DEBUG:
    IMAGE_DIR_PATH = '../input/petfinder-pawpularity-score/train/'
    df_test = pd.read_csv('../input/petfinder-pawpularity-score/train.csv')
    #df_test = df_test.iloc[0:3000]
    df_test = df_test.iloc[0:10]
else:
    IMAGE_DIR_PATH = '../input/petfinder-pawpularity-score/test/'
    df_test = pd.read_csv('../input/petfinder-pawpularity-score/test.csv')

In [ ]:
df_test

In [ ]:
def get_transform(image_size, flip=False):
    if flip:
        valid_aug = A.Compose([
            A.SmallestMaxSize(max_size=image_size, p=1.0),
            A.CenterCrop(height=image_size, width=image_size, p=1.0),
            A.HorizontalFlip(p=1.0),
            A.Normalize(p=1.0),
            ToTensorV2(p=1.0)
        ])
    
    else:
        valid_aug = A.Compose([
            A.SmallestMaxSize(max_size=image_size, p=1.0),
            A.CenterCrop(height=image_size, width=image_size, p=1.0),
            A.Normalize(p=1.0),
            ToTensorV2(p=1.0)
        ])

    return valid_aug




def get_transform2(image_size):
    train_aug = A.Compose([
                           A.Rotate(limit=15, always_apply=True),#p=1),
                           #A.LongestMaxSize(max_size=image_size, p=1.0),
                           A.SmallestMaxSize(max_size=image_size, always_apply=True),
                           A.OneOf([
                                    A.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.1, hue=0.1, p=0.75),# all 0.75
                                    A.RandomBrightnessContrast(brightness_limit=0.2, contrast_limit=0.2, p=0.75),
                                    A.RandomGamma(p=0.75)
                           ], p=1.0),
                           #A.PadIfNeeded(min_height=image_size, min_width=image_size, border_mode=0, p=1.0),
                           A.HorizontalFlip(p=0.5),
                           A.OneOf([
                                    A.CoarseDropout(min_holes=2,
                                                    max_holes=10,
                                                    max_width=image_size//5,
                                                    min_width=image_size//20,
                                                    max_height=image_size//5,
                                                    min_height=image_size//20,
                                                    fill_value=(0,0,0),
                                                    p=0.75),# all 0.75
                                    A.CoarseDropout(min_holes=2,
                                                    max_holes=10,
                                                    max_width=image_size//5,
                                                    min_width=image_size//20,
                                                    max_height=image_size//5,
                                                    min_height=image_size//20,
                                                    fill_value=(127,127,127),
                                                    p=0.75),
                                    A.CoarseDropout(min_holes=2,
                                                    max_holes=10,
                                                    max_width=image_size//5,
                                                    min_width=image_size//20,
                                                    max_height=image_size//5,
                                                    min_height=image_size//20,
                                                    fill_value=(255,255,255),
                                                    p=0.75)
                           ], p=1.0),
                           A.RandomCrop(height=image_size, width=image_size, always_apply=True),
                           A.Normalize(always_apply=True),
                           ToTensorV2(always_apply=True)
    ])
    return train_aug

In [ ]:
class PetfinderDataset(Dataset):
    def __init__(self, df, augs=None):
        self.paths = IMAGE_DIR_PATH + np.array(df['Id']) + '.jpg'
        self.feats = np.array(df[['Subject Focus', 'Eyes', 'Face', 'Near', 'Action', 'Accessory', 'Group', 'Collage', 'Human', 'Occlusion', 'Info', 'Blur']]).astype(np.int)
        self.augs = augs

    def __len__(self):
        return len(self.paths)

    def __getitem__(self, idx):
        image = cv2.imread(self.paths[idx])
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB).astype(np.uint8)
        
        if self.augs:
            image = self.augs(image=image)['image']

        return image, torch.tensor(self.feats[idx], dtype=torch.float)

# model

In [ ]:
class swin_model_WithMeta3(nn.Module):
    def __init__(self, pretrained=True, input_size=None):
        super().__init__()

        if input_size==224:
            self.model = timm.create_model('swin_large_patch4_window7_224', pretrained=pretrained)
        elif input_size==384:
            self.model = timm.create_model('swin_large_patch4_window12_384', pretrained=pretrained)
        else:
            self.model = None

        image_embedding_dim = self.model.head.in_features
        self.model.head = nn.Identity()

        neck_dim = 512#512
        self.fc1 = nn.Linear(image_embedding_dim, neck_dim)
        self.fc2 = nn.Linear(12, neck_dim)

        self.fc3 = nn.Linear(neck_dim, 1)# regression
        self.fc4 = nn.Linear(neck_dim, N_BINS)# classification
        

    def forward(self, input_image, input_features):
        # image feature
        y_image = self.fc1(self.model(input_image))

        # meta feature
        y_feat = torch.sigmoid(self.fc2(input_features))

        # fuse two features
        y = torch.mul(y_image, y_feat)
        y_reg = self.fc3(y)
        y_cls = self.fc4(y)
        
        return y_reg, y_cls

In [ ]:
def sigmoid(x):
    y = 1.0 / (1.0 + np.exp(-x))
    return y

def logit2paw(logit, bin2paw_list):
    probs = softmax(logit, axis=1)
    expects = np.sum(probs*bin2paw_list, axis=1)
    return expects

In [ ]:
def valid_fn(valid_loader, model, device, bin2paw_list):
    model.eval()
    #model.half()#
    
    reg_preds, cls_preds = [], []
    for images, features in tqdm(valid_loader):
        images, features = images.to(device), features.to(device)
        
        #with torch.no_grad():
        with torch.inference_mode():
            reg_pred, cls_pred = model(images, features)
        
        reg_preds.append(reg_pred.to('cpu').detach().numpy())
        cls_preds.append(cls_pred.to('cpu').detach().numpy())
        
    cls_preds = logit2paw(np.concatenate(cls_preds), bin2paw_list)
    reg_preds = sigmoid(np.concatenate(reg_preds))*99 + 1
    
    return reg_preds.squeeze(), cls_preds.squeeze()

In [ ]:
preds = []
former_input_size = 0

for model_path, alpha, bin2paw_list, input_size in zip(MODEL_PATHS, ALPHAS, bin2paw_lists, INPUT_SIZES):
    _=gc.collect()
    
    if former_input_size != input_size:
        # switch model
        model = swin_model_WithMeta3(pretrained=False, input_size=input_size)
        _ = model.to(device)
        print('model switched')
    former_input_size = input_size
        
    
    # model
    print(model_path)
    model.load_state_dict(torch.load(model_path))
    
    # augmentation
    if input_size == 224:
        valid_augs = [get_transform2(input_size)]*N_TTA_224
    elif input_size == 384:
        valid_augs = [get_transform2(input_size)]*N_TTA_384
    else:
        None
    
    preds_TTA = []
    for valid_aug in valid_augs:
        # dataset
        valid_data = PetfinderDataset(df_test.reset_index(drop=True), augs=valid_aug)
        valid_loader = DataLoader(valid_data,
                                  shuffle=False,
                                  num_workers=2,
                                  pin_memory=True,
                                  batch_size=BATCH_SIZE)
        
        
        # predict
        reg_preds, cls_preds = valid_fn(valid_loader, model, device, bin2paw_list)
        pred = alpha*reg_preds + (1.0-alpha)*cls_preds
        preds_TTA.append(pred) # TTAの回数分推論結果を保管
        
    mean_pred_TTA = np.mean(preds_TTA, axis=0) # TTA結果を単純平均
    preds.append(mean_pred_TTA) # モデル毎に結果を保管

preds = np.array(preds)
print(preds.shape)

In [ ]:
# 単純平均
pred_final = np.mean(preds, axis=0)

# LogMeanExp
#pred_final = LogMeanExp(preds, temperature=20)

In [ ]:
df_test['Pawpularity'] = pred_final

In [ ]:
df_submit = df_test[['Id', 'Pawpularity']]
df_submit

In [ ]:
df_submit['Pawpularity'] = df_submit['Pawpularity'].round(1)
df_submit

In [ ]:
df_submit.to_csv("submission.csv", index=False)